In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [2]:
from distgen.generator import Generator
from distgen.plot import plot_2d, plot_1d, plot_current_profile
from matplotlib import pyplot as plt
import yaml

ModuleNotFoundError: No module named 'pint'

In [ ]:
gen = Generator(verbose=0) 
units = {"x":"mm","y":"mm","px":"keV/c","py":"keV/c","t":"ps","q":"pC"}

# Radial Distributions
----------------------

# Radial Uniform

In [ ]:
gen=Generator('data/rad.uniform.in.yaml',verbose=0)
beam = gen.beam()

print("Dictionary input:")
print("r_dist:",gen.input["r_dist"])
print("\nYAML input:")
print(yaml.dump({'r_dist':gen.input["r_dist"]}))

# X-Y Plot
plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=75,axis="equal");

# Radial Gaussian (with Truncation)

In [ ]:
# Truncated radial gaussian distribution with pihole size of 2 mm, gaussian clipped at 50% intensity
gen=Generator('data/rad.trunc.gaussian.in.yaml',verbose=0)
beam = gen.beam()

print("Dictionary input:")
print("r_dist:",gen.input["r_dist"])
print("\nYAML input:")
print(yaml.dump({'r_dist':gen.input["r_dist"]}))

# X-Y Plot
plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=75,axis="equal");

In [ ]:
# Radial gaussian distribution with sigma_xy 2 mm
gen=Generator('data/rad.gaussian.in.yaml',verbose=0)
beam = gen.beam()

print("Dictionary input:")
print("r_dist:",gen.input["r_dist"])

print("\nYAML input:")
print(yaml.dump({'r_dist':gen.input["r_dist"]}))

# X-Y Plot
plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=200,axis="equal");

# Radial File

In [ ]:
# Radial distribution file: a radial gaussian with cuts in it
gen=Generator('data/rad.file.in.yaml', verbose=0)
beam = gen.beam()

print("Dictionary input:")
print("r_dist:",gen.input["r_dist"])
print('\nYAML input:')
print(yaml.dump({'r_dist':gen.input["r_dist"]}))

# X-Y Plot
plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=200,axis="equal");

# Radial Tukey

In [ ]:
# Radial Tukey distribution
gen=Generator('data/tukey.beer.can.in.yaml',verbose=0)
beam = gen.beam()

print("Dictionary Input:")
print("r_dist:", gen.input["r_dist"])
print("\nYAML Input:")
print(yaml.dump({'r_dist':gen.input["r_dist"]}))

plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=100,axis="equal");

# Super Gaussian

In [ ]:
gen=Generator('data/SG.beer.can.in.yaml',verbose=0)
beam = gen.beam()

print("Dictionary Input:")
print("r_dist:", gen.input["r_dist"])
print("\nYAML Input:")
print(yaml.dump({'r_dist':gen.input["r_dist"]}))

plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=100,axis="equal");

# 2D Distributions
------------------

# 2D File

In [ ]:
# 2D XY distribution: Lase Image Example
# Real laser spot image - using 1 million particles (takes a few secs)
gen=Generator('data/laser.image.in.yaml',verbose=0)

beam = gen.beam()

print("Dictionary input:")
print("xy_dist:",gen.input["xy_dist"])

print("\nYAML input:")
print(yaml.dump({'xy_dist':gen.input["xy_dist"]}))

# X-Y Plot
plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=100,axis="equal");

# In the file "laser.image.txt", the first row specfies: x range avg [units], the second the same for y:
# x   3.5002000e+00   0.0000000e+00 [mm]
# y   3.5002000e+00   0.0000000e+00 [mm] 
# The rest of the data is a rectangular equally spaced grid of rho(x,y)

# 1D Distributions
-----------

# Uniform

In [ ]:
gen=Generator('data/x.uniform.y.file.t.gaussian.in.yaml',verbose=0)
beam = gen.beam()

print("Dictionary input:")
print("x_dist:", gen.input["x_dist"])
print("\nYAML Input:")
print(yaml.dump({'x_dist':gen.input["x_dist"]}))

plot_1d(beam,"x",units["x"],nbins=100)

# 1D File

In [ ]:
print("Dictionary input:")
print("y_dist:",gen.input["y_dist"])
print("\nYAML Input:")
print(yaml.dump({'y_dist':gen.input["y_dist"]}))
plot_1d(beam,"y",units["y"],nbins=50)

# Gaussian (with Truncation)

In [ ]:
print("Dictionary Input:")
print("t_dist:",gen.input["t_dist"])
print("\nYAML Input:")
print(yaml.dump({'t_dist':gen.input["t_dist"]}))

plot_1d(beam,"t",units["t"],nbins=100)

# Tukey 

In [ ]:
# Tukey distribution
gen=Generator('data/tukey.beer.can.in.yaml', verbose=0)
beam = gen.beam()

print("User Input:")
print("t_dist:",gen.input["t_dist"])
print("\nYAML Input:")
print(yaml.dump({'t_dist':gen.input["t_dist"]}))

plot_current_profile(beam,1,units)

In [ ]:
# Super Gaussian distribution
gen=Generator('data/SG.beer.can.in.yaml', verbose=0)
beam = gen.beam()

print("User Input:")
print("t_dist:",gen.input["t_dist"])
print("\nYAML Input:")
print(yaml.dump({'t_dist':gen.input["t_dist"]}))

plot_current_profile(beam,1,units)

# Special Temporal Distributions
-------

# Pulse stacking temporal crystals

In [ ]:
gen=Generator('data/t.crystals.in.yaml',verbose=0)
beam = gen.beam()
print("Dictionary Input:")
print("t_dist:",gen.input["t_dist"])
print("\nYAML Input:")
print(yaml.dump({'t_dist':gen.input["t_dist"]}))
plot_current_profile(beam,1,units)